In [8]:
!pip install PyDictionary

    100% |████████████████████████████████| 71kB 4.5MB/s 
  Running setup.py bdist_wheel for PyDictionary ... - done
  Stored in directory: /content/.cache/pip/wheels/8f/55/4a/14dc8470ed285bf2463f958022322e8de7c46c2555c45e3007
  Running setup.py bdist_wheel for goslate ... - done
  Stored in directory: /content/.cache/pip/wheels/4f/7f/28/6f52271012a7649b54b1a7adaae329b4246bbbf9d1e4f6e51a
  Running setup.py bdist_wheel for futures ... - \ done
  Stored in directory: /content/.cache/pip/wheels/f3/f9/c7/4fbf1faa6038faf183f6e3ea61f17a5f7eea5ab9a1dd7753fd
Successfully built PyDictionary goslate futures


# Explanation
First we need to install TensorFlow in the machine. Then `import` them.


In [0]:
import requests
from contextlib import closing
import codecs
import csv
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from PyDictionary import PyDictionary
pydict = PyDictionary()
import urllib.request as urllib

# Reproducability :D 
np.random.seed(7)

randomTextURL = 'https://raw.githubusercontent.com/carykh/neuralNetworkLanguageDetection/master/data/output0.txt'
keyMashTextURL = 'https://raw.githubusercontent.com/carykh/neuralNetworkLanguageDetection/master/data/output1.txt'
#bigNotGibberishURL = 'https://raw.githubusercontent.com/rrenaud/Gibberish-Detector/master/big.txt'

randomURLResponse = requests.get(randomTextURL)
keyMashURLResponse = requests.get(keyMashTextURL)
#bigNotGibberishURLResponse = request.urlopen(bigNotGibberishURL)

randomText = csv.reader(randomURLResponse.text)
keyMashText = csv.reader(keyMashURLResponse.text)

In [11]:
data_words, data_labels = [], []

with requests.get(randomTextURL).text as randomText:
  
  for row in randomText:
    data_words.append(row[0])
    data_labels.append('Random Text')
  
for row in keyMash:
  data_words.append(row[0])
  data_labels.append('Keyboard Mash')
  
#for word in bigNotGibberish:
#  pass # something
  
# lets take 80% data as training and remaining 20% for test.
train_size = int(len(data) * .8)
 
train_words = data_words[:train_size]
train_labels = data_labels[:train_size]

test_words = data_words[train_size:]
test_labels = data_labels[train_size:]
  
# 2 different types
num_labels = 2
vocab_size = 15000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_posts)
 
x_train = tokenizer.texts_to_matrix(train_words, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_words, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

AttributeError: ignored

In [0]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

In [0]:
# Given a single, one-line string, determine if it contains an english word or
# not. If it does, determine whether this random text was produced via keyboard
# mashing or if it was randomly generated by a computer. Keyboard mashing likely
# contains letters around a certain key: if one key is pressed, as well as 6
# surrounding keys (at least 4), and this pattern appears for 75% of all of the
# random text, then it was probably mashed.

def determine_gibberish(line):
  gibberish = True
  gibberish_count = 0
  total_count = 0
  words = line.split()
  for word in words:
    if pydict.meaning(word) is None:
      gibberish_count+=1
      total_count+=1
    else:
      total_count+=1
      
  if gibberish_count / total_count < 0.25:
    gibberish = False
    
  return gibberish

def percent_gibberish(line):
  gibberish = True
  gibberish_count = 0
  total_count = 0
  words = line.split()
  for word in words:
    if pydict.meaning(word) is None:
      gibberish_count+=1
      total_count+=1
    else:
      total_count+=1
      
  return gibberish_count / total_count
  

In [0]:
def determine_mashing(line):
  # assumes line is gibberish
  # key set only includes the alphabet, not the other keys on the keyboard because i'm lazy
  common_keys = {"Q":["W","A"],"W":["Q","A","S","D","E"], "E":["W","S","D","F","R"], "R":["E","D","F","G","T"],"T":["R","F","G","H","Y"],
                "Y":["T","G","H","J","Y"], "U":["Y","H","J","I","K"], "I":["U","J","K","L","O"], "O":["I","K","L","P"],
                "A":["Q","W","S","Z"],"S":["Q","W","E","D","X","Z","A"],"D":["W","E","R","F","C","X","S"],"F":["D","E","R","T","G","V","C"],
                "G":["R","T","Y","H","B","V","F"],"H":["T","Y","U","J","N","B","G"],"J":["Y","U","I","K","M","N","H"],
                "K":["U","I","O","L","M","J"],"L":["I","O","P","K","M"],"Z":["A","S","X"],"X":["Z","S","D","C"],"C":["X","D","F","V"],
                "V":["C","F","G","B"],"B":["V","G","H","N"],"N":["B","H","J","M"],"M":["N","J","K","L",","]}
  
  mashing = False
  mash_count = 0
  total_count = 0
  words = str(line.split())
  letters = list(words)

  for word in words: # I think this might not work because it doesn't move on to another word until it checked the same thing a bunch of times
    if words.index(word) + 3 is not None and words.index(word) - 3 is not None:
      for letter in letters:   
        if ((letters.index(letter) in common_keys and (letters[letters.index(letter) + 1] in common_keys
                                                    and letters[letters.index(letter) + 2] in common_keys
                                                    and letters[letters.index(letter) + 3] in common_keys)) 
        or (letters.index(letter) in common_keys and (letters[letters.index(letter) - 1] in common_keys
                                                     and letters[letters.index(letter) - 2] in common_keys
                                                     and letters[letters.index(letter) - 3] in common_keys))):
          mash_count+=1
          total_count+=1
    elif words.index(word) + 2 is not None and words.index(word) - 2 is not None:
      for letter in letters:
        if ((letters.index(letter) in common_keys and (letters[letters.index(letter) + 1] in common_keys
                                                     and letters.index(letter) + 2 in common_keys))
        or (letters.index(letter) in common_keys and (letters.index(letter) - 1 in common_keys
                                                     and letters.index(letter) - 2 in common_keys))):
          mash_count+=1
          total_count+=1
    elif words.index(word) + 1 is not None and words.index(word) - 1 is not None:
      for letter in letters:
        if ((letters.index(letter) in common_keys and (letters.index(letter) + 1 in common_keys))
        or (letters.index(letter) in common_keys and (letters.index(letter) - 1 in common_keys))):
          mash_count+=1
          total_count+=1
    else:
      total_count+=1
      
  if (mash_count/total_count > 0.75):
    mashing = True
    
  return mashing
          

In [19]:
# Prompt the user for input for a file, examine the file line by line, and determine
# 1) whether or not it is gibberish, as well as the percent gibberish,
# and 2) if it is gibberish, the percent mashed

if __name__ == "__main__":
  gibberish_count = 0
  total_count = 0
  mashing_count = 0
  total_mashing_count = 0
  
  file = input("Enter the full link to the file you wish to examine: ")
  data = urllib.urlopen(file)
  for line in data:
    if determine_gibberish(line):
        gibberish_count+=1
        total_count+=1
    else:
      total_count+=1

    if determine_mashing(line):
      mashing_count+=1
      total_mashing_count+=1
    else:
      total_mashing_count+=1
        
  calculated_gibberish = gibberish_count / total_count
  calculated_mashing = mashing_count / total_mashing_count
  print("Your file is " + calculated_gibberish*100 + "% gibberish")
    
  if calculated_gibberish >= 0.75 and calculated_mashing >= 0.75:
    print("Your file is determined to be gibberish and the result of keyboard mashing")
  elif calculated_gibberish >= 0.75 and calculated_mashing < 0.75:
    print("Your file is determined to be gibberish and the result of some other form of random generation -- not keyboard mashing")
  else:
    print("Your file is determined to not be gibberish")

Enter the full link to the file you wish to examine: https://raw.githubusercontent.com/carykh/neuralNetworkLanguageDetection/master/data/output1.txt
Error: The Following Error occured: list index out of range


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


ZeroDivisionError: ignored